In [15]:
import requests
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
import klib
from pint import UnitRegistry
# First we create the registry.
ureg = UnitRegistry()
# Then we append the new definitions
ureg.load_definitions('crypto_mining_units.txt')
pd.set_option("display.max.columns", None)


In [16]:
def get_network_data(asset: str, metrics: list = ['HashRate', 'IssTotUSD', 'FeeTotUSD', 'PriceUSD', 'FeeMeanNtv'], end_time: str = date.today().strftime('%Y-%m-%d')):
    api_str = f"https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?assets={asset}&metrics={','.join(metrics)}&end_time={end_time}&page_size=10000&pretty=true"
    response = requests.get(api_str).json()
    return pd.DataFrame(response['data'])

In [17]:
df_unclean = get_network_data("ltc", ['HashRate', 'PriceUSD', 'FeeMeanNtv']).fillna("0")

df_cleaned = klib.data_cleaning(df_unclean)

df_cleaned.time = df_cleaned.time.str.replace("T00:00:00.000000000Z", "")
df_cleaned.time = pd.to_datetime(df_cleaned.time)

df_cleaned[['hash_rate', 'price_usd']] =  df_cleaned[['hash_rate', 'price_usd']].apply(pd.to_numeric)

df_cleaned.index = df_cleaned.time
df_cleaned.drop(columns=['time'], inplace=True)

df_cleaned.dtypes
df = df_cleaned
df

Shape of cleaned data: (3559, 4)Remaining NAs: 0

Changes:
Dropped rows: 0
     of which 0 duplicates. (Rows: [])
Dropped columns: 1
     of which 1 single valued.     Columns: ['asset']
Dropped missing values: 0
Reduced memory by at least: -0.15 MB (--107.14%)



,fee_mean_ntv,hash_rate,price_usd
time,,,
2011-10-08,0,2.427259e-11,0.000000
2011-10-12,0,2.427259e-11,0.000000
2011-10-13,0.0000108695652173913,1.801158e-06,0.000000
2011-10-14,0.000032258064516129,2.577167e-06,0.000000
2011-10-15,0.000178571428571429,2.367452e-06,0.000000
...,...,...,...
2021-07-04,0.00011324,2.351037e+02,145.227147
2021-07-05,0.00015587,2.306762e+02,138.805518
2021-07-06,0.00019893,2.345860e+02,138.514313


In [63]:
# how many days it takes to pay off the machine
# D is the static days-to-breakeven
# cost of histing and machines
# C is the upfront capital expenditure 
# P is the current Litecoin price
# S is hashrate produced by the purchased equipments
# H is the network hashrate
# m is the coinbase reward, currently it is 6.25BTC.
# n is the current avg. transaction fee per block
# k is the efficiency (J/T) of the equipments
# r is the all-in electricity cost ($/KwH)
# C = 5_000
# P = 150
# Goldshell LT5 Pro MH/s
# S = 2050
# H = 216 * 1000000
# m = 100
# n = 1
# k = 0.00079
# r = 0.1
# Bitmain L7 | 9.5GH@3425W = .002774 rating (360W per GH)
# Goldshell LT5 | 2.05GH@2150W = .000953 rating (1048W per GH)
# Goldshell LT5 Pro | 2.45@3100W = .00079 rating (1265W per GH)
# days to break even mining litecoin on a goldshell LT5 Pro
# denominator = P * (S/H) * (m + n) * 6 * 24 - k * S * r * 24
# print(C/denominator)

297.29697589516127


In [29]:
# Calculating days to break even for one G LT5 Pro
C = 5_000 * ureg.USD
P = 150 * ureg.USD
miner_hashrate = ureg.TH / ureg.second
# Goldshell LT5 Pro MH/s
S = 0.00205 * miner_hashrate
network_hashrate = ureg.TH / ureg.second
H = 216 * network_hashrate
# coinbase
m = 12.5 * ureg.LTC
n = 0.0001984 * ureg.LTC
k = 0.00079 * ureg.J / ureg.TH
r = 0.1 * (ureg.USD / ureg.kWh)
D = ureg.day

share_of_hashrate = S/H

block_reward = m + n



print(C / (P * share_of_hashrate * block_reward * 24 * 24 - k * S * r * 24))


DimensionalityError: Cannot convert from 'LTC * USD' ([currency] ** 2) to 'USD * joule / kilowatt_hour / second' ([currency] / [time])